import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.graph_objects as go
import re

# ---------------------------------------------------------
# AUTO-DETECT USER COUNTRY VIA IP
# ---------------------------------------------------------
try:
    geo_resp = requests.get("https://ipapi.co/json/", timeout=5).json()
    USER_COUNTRY = geo_resp.get("country_name")
except Exception:
    USER_COUNTRY = None

print("Detected country:", USER_COUNTRY)

# ---------------------------------------------------------
# FETCH NATIONAL DEBT DATA
# ---------------------------------------------------------
url = "https://worldpopulationreview.com/country-rankings/countries-by-national-debt"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

table = soup.find("table")
rows = table.find_all("tr")[1:]

# multipliers for debt units
DEBT_MULT = {"T": 1e12, "B": 1e9, "M": 1e6}

records = []

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 5:
        continue

    country = cols[1].text.strip()
    if country.upper() == "TOTAL":
        continue

    debt_text = cols[2].text.strip()
    gdp_pct_text = cols[3].text.strip()
    per_capita_text = cols[4].text.strip()

    # get the debt in USD
    m = re.match(r"\$(\d+(\.\d+)?)([TBM])", debt_text)
    if not m:
        continue
    debt_val = float(m.group(1))
    debt_unit = m.group(3)
    debt_usd = int(debt_val * DEBT_MULT[debt_unit])

    # get debt as % of GDP
    debt_pct_gdp = float(gdp_pct_text.replace("%", "")) if "%" in gdp_pct_text else None

    # get debt per person
    per_capita = None
    pc_text = per_capita_text.replace("$", "").replace(",", "").strip()
    pc_match = re.match(r"(\d+(\.\d+)?)\s*(Mn|Bn)?", pc_text)
    if pc_match:
        pc_val = float(pc_match.group(1))
        pc_unit = pc_match.group(3)
        if pc_unit == "Mn":
            per_capita = int(pc_val
